<a href="https://colab.research.google.com/github/BramanTyamahedrawan/Machine-Learning-SMT-5/blob/main/pertemuan-6/Tugas_Praktikum_Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TUGAS PRAKTIKUM ENSEMBLE LEARNING**

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **TUGAS 1**

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [25]:
# Import data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB # import Naive Bayes model Gaussian
from sklearn.svm import SVC # import SVM classifier
from sklearn.ensemble import VotingClassifier # import model Voting

# Baca dataset mushroom (pastikan file dataset ada dalam direktori yang sama)
data = pd.read_csv('/content/drive/MyDrive/ML/Pertemuan 6/mushrooms.csv')

In [26]:
# Cek kolom null
data.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [27]:
# Encode label kategorikal menjadi numerik
le = LabelEncoder()
data['class'] = le.fit_transform(data['class'])
data = pd.get_dummies(data, drop_first=True)

In [28]:
# Pisahkan data menjadi fitur dan target
X = data.drop('class', axis=1)
y = data['class']

# Pisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Decision Tree
dt_classifier = DecisionTreeClassifier()
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy']
}
grid_search_dt = GridSearchCV(dt_classifier, param_grid, cv=5)
%time grid_search_dt.fit(X_train, y_train)

CPU times: user 1.23 s, sys: 0 ns, total: 1.23 s
Wall time: 1.26 s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30]})

In [29]:
best_dt_model = grid_search_dt.best_estimator_
%time y_pred_dt = best_dt_model.predict(X_test)
# Evaluasi Pohon Keputusan
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)
print(classification_report(y_test, y_pred_dt))

CPU times: user 3.43 ms, sys: 0 ns, total: 3.43 ms
Wall time: 3.41 ms
Decision Tree Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



In [31]:
# RandomForest
rf_classifier = RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2']
}
grid_search_rf = GridSearchCV(rf_classifier, param_grid, cv=5)
%time grid_search_rf.fit(X_train, y_train)

CPU times: user 52.2 s, sys: 135 ms, total: 52.3 s
Wall time: 53.5 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100, 200]})

In [32]:
best_rf_model = grid_search_rf.best_estimator_
%time y_pred_rf = best_rf_model.predict(X_test)
# Evaluasi RandomForest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("RandomForest Accuracy:", accuracy_rf)
print(classification_report(y_test, y_pred_rf))

CPU times: user 17 ms, sys: 968 µs, total: 17.9 ms
Wall time: 22 ms
RandomForest Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



# **TUGAS 2**

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [33]:
# Import data
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [34]:
# Decision Tree
dt_classifier = DecisionTreeClassifier()
param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy']
}
grid_search_dt = GridSearchCV(dt_classifier, param_grid_dt, cv=5)
grid_search_dt.fit(X_train, y_train)
best_dt_model = grid_search_dt.best_estimator_
y_pred_dt = best_dt_model.predict(X_test)

In [35]:
# AdaBoost
adaboost_classifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), random_state=42)
param_grid_adaboost = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0]
}
grid_search_adaboost = GridSearchCV(adaboost_classifier, param_grid_adaboost, cv=5)
grid_search_adaboost.fit(X_train, y_train)
best_adaboost_model = grid_search_adaboost.best_estimator_
y_pred_adaboost = best_adaboost_model.predict(X_test)

In [36]:
# Evaluasi Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)
print(classification_report(y_test, y_pred_dt))

Decision Tree Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



In [37]:
# Evaluasi AdaBoost
accuracy_adaboost = accuracy_score(y_test, y_pred_adaboost)
print("AdaBoost Accuracy:", accuracy_adaboost)
print(classification_report(y_test, y_pred_adaboost))

AdaBoost Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



# **Tugas 3**

Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma

1. Logistic Regression
2. SVM kernel polynomial
3. Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter

In [38]:
# Import dan load data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV

# Load the diabetes dataset
data = pd.read_csv('/content/drive/MyDrive/ML/Pertemuan 6/diabetes.csv')

In [39]:
# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(data.loc[data[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


In [40]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

data[feature_columns] = fill_values.fit_transform(data[feature_columns])

In [41]:
# Split data into fitur and target
X = data[feature_columns]
y = data.Outcome

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Buat model dasar dengan penyetelan hyperparameter (Anda dapat menyesuaikannya)
logistic_regression = LogisticRegression()
svm_poly = SVC(kernel='poly')
decision_tree = DecisionTreeClassifier()

param_grid_svm = {
    'C': [0.1, 1, 10],
    'degree': [2, 3, 4]
}

param_grid_dt = {
    'max_depth': [None, 10, 20, 30],
    'criterion': ['gini', 'entropy']
}

grid_search_svm = GridSearchCV(svm_poly, param_grid_svm, cv=5)
grid_search_dt = GridSearchCV(decision_tree, param_grid_dt, cv=5)

grid_search_svm.fit(X_train, y_train)
grid_search_dt.fit(X_train, y_train)

best_svm = grid_search_svm.best_estimator_
best_dt = grid_search_dt.best_estimator_

# Buat pengklasifikasi pemungutan suara ansambel
ensemble_classifier = VotingClassifier(estimators=[
    ('logistic', logistic_regression),
    ('svm', best_svm),
    ('decision_tree', best_dt)
], voting='hard')

# Sesuaikan pengklasifikasi ansambel dengan data pelatihan
ensemble_classifier.fit(X_train, y_train)

# Evaluasi pengklasifikasi ansambel pada data pengujian
ensemble_accuracy = ensemble_classifier.score(X_test, y_test)
print("Ensemble Voting Accuracy:", ensemble_accuracy)

Ensemble Voting Accuracy: 0.7597402597402597
